In [34]:
# Import netket library
import netket as nk
# Helper libraries
import numpy as np
import itertools
import matplotlib.pyplot as plt

In [41]:
L = 2

g = nk.graph.Hypercube(length=L, n_dim=1, pbc=False)
print(g.edges)
# Spin based Hilbert Space
hi = nk.hilbert.Spin(s=1/2, N=g.n_nodes)

<bound method NetworkX.edges of Grid(length=[2], pbc=False)>


In [44]:
OB = np.load('../data/integrals/STO-3G/STO-3G_H2_OB_d0-734_eq1.npy')
TB = np.load('../data/integrals/STO-3G/STO-3G_H2_TB_d0-734_eq1.npy')

In [43]:
def SB(i,j):
    return 1
def TB(i,j,k,l):
    return 1

In [45]:
# Pauli Matrices
sigmaz = np.array([[1, 0], [0, -1]])
sigmax = np.array([[0, 1], [1, 0]])
isigmay = np.array([[0, 1], [-1, 0]])

sigma_p = (sigmax + isigmay)/2
sigma_m = (sigmax - isigmay)/2

In [46]:
from functools import lru_cache
@lru_cache(maxsize=128)
def jw(i):
    return 1 if i==0 else np.kron(jw(i-1), sigmaz)

In [47]:
jw(i=10)

array([[ 1,  0,  0, ...,  0,  0,  0],
       [ 0, -1,  0, ...,  0,  0,  0],
       [ 0,  0, -1, ...,  0,  0,  0],
       ...,
       [ 0,  0,  0, ..., -1,  0,  0],
       [ 0,  0,  0, ...,  0, -1,  0],
       [ 0,  0,  0, ...,  0,  0,  1]])

In [48]:
%%time
from netket.operator import LocalOperator as Op
ha = Op(hi)
for i,j in itertools.product(range(L), range(L)):
    #print(i,j)
    i_part = np.kron(jw(i), sigma_p)
    if i<j: i_part = np.kron(i_part, np.eye(2**(j-i)))
    j_part = np.kron(jw(j), sigma_m)
    if j<i: j_part = np.kron(j_part, np.eye(2**(i-j)))

    operator = OB[i,j] * (i_part @ j_part)
    sites = [i for i in range(max(i,j)+1)]
    ha += Op(hi, operator, sites)

for i,j,k,l in itertools.product(range(L), repeat=4):
    #print(i,j,k,l)
    m = max(i,j,k,l)
    i_part = np.kron(jw(i), sigma_p)
    if i<m: i_part = np.kron(i_part, np.eye(2**(m-i)))
    j_part = np.kron(jw(j), sigma_m)
    if j<m: j_part = np.kron(j_part, np.eye(2**(m-j)))
    k_part = np.kron(jw(k), sigma_p)
    if k<m: k_part = np.kron(k_part, np.eye(2**(m-k)))
    l_part = np.kron(jw(l), sigma_m)
    if l<m: l_part = np.kron(l_part, np.eye(2**(m-l)))

    operator = TB[i,j,k,l] * (i_part @ k_part @ l_part @ j_part)
    sites = [i for i in range(max(i,j,k,l)+1)]
    ha += Op(hi, operator, sites)

CPU times: user 18.8 ms, sys: 1.11 ms, total: 19.9 ms
Wall time: 21.8 ms


In [49]:
print(ha.to_dense())
print(hi.size)
print(hi.n_states)

[[-1.70840817  0.          0.          0.        ]
 [ 0.         -1.12487186  0.96643592  0.        ]
 [ 0.          0.96643592 -1.12487186  0.        ]
 [ 0.          0.          0.          0.        ]]
2
4


In [50]:
res = nk.exact.lanczos_ed(ha, k=1, compute_eigenvectors=False)
print("Exact ground state energy = {0:.3f}".format(res[0]))

Exact ground state energy = -2.091


In [51]:
for perm in itertools.permutations('ijkl'):
    OB = np.load('../data/integrals/STO-3G/STO-3G_H2_OB_d0-734_eq1.npy')
    TB = np.load('../data/integrals/STO-3G/STO-3G_H2_TB_d0-734_eq1.npy')
    TB = np.einsum('ijkl->' + ''.join(perm), TB)
    from netket.operator import LocalOperator as Op
    ha = Op(hi)
    for i,j in itertools.product(range(L), range(L)):
        #print(i,j)
        i_part = np.kron(jw(i), sigma_p)
        if i<j: i_part = np.kron(i_part, np.eye(2**(j-i)))
        j_part = np.kron(jw(j), sigma_m)
        if j<i: j_part = np.kron(j_part, np.eye(2**(i-j)))

        operator = OB[i,j] * (i_part @ j_part)
        sites = [i for i in range(max(i,j)+1)]
        ha += Op(hi, operator, sites)

    for i,j,k,l in itertools.product(range(L), repeat=4):
        #print(i,j,k,l)
        m = max(i,j,k,l)
        i_part = np.kron(jw(i), sigma_p)
        if i<m: i_part = np.kron(i_part, np.eye(2**(m-i)))
        j_part = np.kron(jw(j), sigma_m)
        if j<m: j_part = np.kron(j_part, np.eye(2**(m-j)))
        k_part = np.kron(jw(k), sigma_p)
        if k<m: k_part = np.kron(k_part, np.eye(2**(m-k)))
        l_part = np.kron(jw(l), sigma_m)
        if l<m: l_part = np.kron(l_part, np.eye(2**(m-l)))

        operator = TB[i,j,k,l] * (i_part @ k_part @ l_part @ j_part)
        sites = [i for i in range(max(i,j,k,l)+1)]
        ha += Op(hi, operator, sites)
    res = nk.exact.lanczos_ed(ha, k=1, compute_eigenvectors=False)
    print("Exact ground state energy (switch {})= {}".format(''.join(perm), res[0]))

Exact ground state energy (switch ijkl)= -2.0913077833799796
Exact ground state energy (switch ijlk)= -2.0913077833799796
Exact ground state energy (switch ikjl)= -2.2497437216475116
Exact ground state energy (switch iklj)= -2.791079273212312
Exact ground state energy (switch iljk)= -2.2497437216475116
Exact ground state energy (switch ilkj)= -2.7910792732123118
Exact ground state energy (switch jikl)= -2.0913077833799796
Exact ground state energy (switch jilk)= -2.0913077833799796
Exact ground state energy (switch jkil)= -2.249743721647512
Exact ground state energy (switch jkli)= -2.7910792732123113
Exact ground state energy (switch jlik)= -2.249743721647512
Exact ground state energy (switch jlki)= -2.7910792732123113
Exact ground state energy (switch kijl)= -2.7910792732123118
Exact ground state energy (switch kilj)= -2.249743721647512
Exact ground state energy (switch kjil)= -2.7910792732123118
Exact ground state energy (switch kjli)= -2.2497437216475116
Exact ground state energy (s